In [1]:
from tqdm import tqdm
import requests
import zipfile
import os
from io import BytesIO
import pandas as pd
from tabulate import tabulate
import chardet
import itertools
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [2]:
LAT = 51.103333
LON = 16.9

START_DATE = "2016-01-01"
END_DATE = "2025-11-30"
OUTPUT_DIR = "./data"
TEMP_DIR = "./.tmp"

In [ ]:
def get_data_from_meteo(lat, lon, from_date, to_date):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    #! website: https://open-meteo.com/en/docs/historical-weather-api?hourly=temperature_2m,relative_humidity_2m,apparent_temperature,surface_pressure,wind_direction_10m,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,rain,dew_point_2m,snowfall,snow_depth,cloud_cover&location_mode=csv_coordinates
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": from_date,
        "end_date": to_date,
        "timezone": "Europe/Berlin",
        "hourly": ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "surface_pressure", "wind_direction_10m", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "rain", "dew_point_2m", "snowfall", "snow_depth", "cloud_cover"],
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    # print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
    # print(f"Elevation: {response.Elevation()} m asl")
    # print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(3).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()
    hourly_soil_temperature_0_to_7cm = hourly.Variables(6).ValuesAsNumpy()
    hourly_soil_moisture_0_to_7cm = hourly.Variables(7).ValuesAsNumpy()
    hourly_rain = hourly.Variables(8).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(9).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(10).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(12).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
    hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
    hourly_data["rain"] = hourly_rain
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["cloud_cover"] = hourly_cloud_cover

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe

In [4]:
os.makedirs(OUTPUT_DIR, exist_ok = True)
os.makedirs(TEMP_DIR, exist_ok = True)

In [5]:
start = pd.to_datetime(START_DATE)
end = pd.to_datetime(END_DATE)
months = pd.date_range(start=start, end=end, freq='MS')

for month_start in tqdm(months, desc="Processing months"):
    output_filename = os.path.join(TEMP_DIR, f"weather_data_{month_start.strftime('%Y_%m')}.csv")
    if os.path.exists(output_filename):
        continue
        
    month_end = (month_start + pd.offsets.MonthEnd(1)).normalize()
    actual_start = max(start, month_start)
    actual_end = min(end, month_end)

    df = get_data_from_meteo(LAT, LON, month_start.strftime("%Y-%m-%d"), month_end.strftime("%Y-%m-%d"))
    df.to_csv(output_filename, index=False)
    
try:
    os.remove('.cache.sqlite')
except Exception as e:
    print(e)

Processing months:   0%|          | 0/119 [00:00<?, ?it/s]

Processing months: 100%|██████████| 119/119 [02:17<00:00,  1.15s/it]

[WinError 32] The process cannot access the file because it is being used by another process: '.cache.sqlite'


In [6]:
files = os.listdir(TEMP_DIR)
df = pd.concat([pd.read_csv(os.path.join(TEMP_DIR, f)) for f in tqdm(files, desc="Combining files")], ignore_index=True)

df.dropna(inplace=True)
df['date'] = pd.to_datetime(df['date'])

Combining files: 100%|██████████| 119/119 [00:03<00:00, 32.10it/s]


In [7]:
print(f"Shape: {df.shape}")
print(tabulate(df.tail(20), headers='keys', tablefmt='psql'))


Shape: (86928, 14)
+-------+---------------------------+------------------+------------------------+------------------------+--------------------+----------------------+------------------+-----------------------------+--------------------------+--------+----------------+------------+--------------+---------------+
|       | date                      |   temperature_2m |   relative_humidity_2m |   apparent_temperature |   surface_pressure |   wind_direction_10m |   wind_speed_10m |   soil_temperature_0_to_7cm |   soil_moisture_0_to_7cm |   rain |   dew_point_2m |   snowfall |   snow_depth |   cloud_cover |
|-------+---------------------------+------------------+------------------------+------------------------+--------------------+----------------------+------------------+-----------------------------+--------------------------+--------+----------------+------------+--------------+---------------|
| 86908 | 2025-11-30 03:00:00+00:00 |            -1.45 |                98.9045 |         

In [8]:
from_date = df['date'].max().date()
to_date = df['date'].min().date()
df.sort_values(by='date', inplace=True)
df.to_csv(os.path.join(OUTPUT_DIR, f'meteo_{LON}_{LAT}_{from_date}_{to_date}.csv'), index=False)


In [9]:
for f in os.listdir(TEMP_DIR):
    os.remove(os.path.join(TEMP_DIR, f))
os.rmdir(TEMP_DIR)